In [2]:
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string 
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
import numpy as np

Using TensorFlow backend.


In [3]:
# load data into list
l_text = open("dataset-1/dialogues_text.txt").readlines()
l_act = open("dataset-1/dialogues_act.txt").readlines()
l_emotion = open("dataset-1/dialogues_emotion.txt").readlines()
l_topic = open("dataset-1/dialogues_topic.txt").readlines()
len(l_topic)

13118

In [4]:
#Clean function
ps = nltk.PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words= stopwords.words('english')

def clean_text(text):
    text_rc1 = re.sub('[0-9]+', '', text)
    text_rc2 = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|''[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text_rc1)
    text = re.sub('eou+', '', text_rc2)
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    tokens = re.split('\W+', text_lc)    # tokenization
    #text = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    #text = [ps.stem(word) for word in text1 if word not in stop_words]  # remove stopwords and stemming
    return tokens

In [5]:
act_list1=[j for i in l_act for j in i]
act_lst = list(map(str.strip, act_list1))
act_list=[x for x in act_lst if x]
len(act_list)

102979

In [6]:
emo_list1=[j for i in l_emotion for j in i]
emo_lst = list(map(str.strip, emo_list1))
emo_list=[x for x in emo_lst if x]
len(emo_list)

102979

In [7]:
sentences_list1=[x for xs in l_text for x in xs.split('__eou__')]
sentence_lst = list(map(str.strip, sentences_list1))
sentences=[x for x in sentence_lst if x]
len(sentences)

102980

In [8]:
n = {'Text':l_text,'Topic':l_topic,'Emotions':l_emotion,'Act':l_act}
df = pd.DataFrame(n)
df= df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
df.columns = ['act', 'emotion','text','topic']
df.head()

,act,emotion,text,topic
0,The kitchen stinks . __eou__ I'll throw out th...,1,2 0,3 4
1,"So Dick , how about getting some coffee for to...",1,4 2 0 1 0,3 4 3 1 1
2,Are things still going badly with your housegu...,1,0 1 0 0,2 1 3 4
3,Would you mind waiting a while ? __eou__ Well ...,1,0 0 0 4,3 2 1 1
4,Are you going to the annual party ? I can give...,1,0 4 4,3 4 1


In [9]:
#Creating dataframe of Conversations with Topic id
n1 = {'Text':l_text,'Topic':l_topic}
n1
df_TextnTopic = pd.DataFrame(n1)
df1 = df_TextnTopic.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
df1.head()

,Text,Topic
0,The kitchen stinks . __eou__ I'll throw out th...,1
1,"So Dick , how about getting some coffee for to...",1
2,Are things still going badly with your housegu...,1
3,Would you mind waiting a while ? __eou__ Well ...,1
4,Are you going to the annual party ? I can give...,1


In [61]:
#adding new column to the df with clean text
df1['clean'] = df1['Text'].apply(lambda x: clean_text(x))
df1['text_string'] = [','.join(map(str, l)) for l in df1['clean']]
df1.head()

,Text,Topic,clean,texr_string,text_string
0,The kitchen stinks . __eou__ I'll throw out th...,1,"[the, kitchen, stinks, ill, throw, out, the, g...","the,kitchen,stinks,ill,throw,out,the,garbage,","the,kitchen,stinks,ill,throw,out,the,garbage,"
1,"So Dick , how about getting some coffee for to...",1,"[so, dick, how, about, getting, some, coffee, ...","so,dick,how,about,getting,some,coffee,for,toni...","so,dick,how,about,getting,some,coffee,for,toni..."
2,Are things still going badly with your housegu...,1,"[are, things, still, going, badly, with, your,...","are,things,still,going,badly,with,your,housegu...","are,things,still,going,badly,with,your,housegu..."
3,Would you mind waiting a while ? __eou__ Well ...,1,"[would, you, mind, waiting, a, while, well, ho...","would,you,mind,waiting,a,while,well,how,long,w...","would,you,mind,waiting,a,while,well,how,long,w..."
4,Are you going to the annual party ? I can give...,1,"[are, you, going, to, the, annual, party, i, c...","are,you,going,to,the,annual,party,i,can,give,y...","are,you,going,to,the,annual,party,i,can,give,y..."


In [62]:
x_train, x_test, y_train, y_test = train_test_split(df1["text_string"],df1["Topic"], test_size = 0.2, random_state = 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(Counter(y_train))
print(Counter(y_test))

(10494,)
(10494,)
(2624,)
(2624,)
Counter({'5': 3515, '1': 2886, '8': 1555, '6': 854, '4': 529, '10': 395, '2': 386, '7': 213, '9': 117, '3': 44})
Counter({'5': 876, '1': 760, '8': 369, '6': 211, '4': 138, '2': 101, '10': 90, '7': 54, '9': 14, '3': 11})


In [63]:
results_df = pd.DataFrame()

In [64]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

from sklearn.metrics import classification_report
y_pred1 = nb.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred1, y_test))
print(classification_report(y_test, y_pred1))

accuracy 0.5659298780487805
              precision    recall  f1-score   support

           1       0.57      0.72      0.64       760
          10       0.00      0.00      0.00        90
           2       0.00      0.00      0.00       101
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00       138
           5       0.53      0.93      0.68       876
           6       1.00      0.00      0.01       211
           7       0.00      0.00      0.00        54
           8       0.87      0.33      0.48       369
           9       0.00      0.00      0.00        14

   micro avg       0.57      0.57      0.57      2624
   macro avg       0.30      0.20      0.18      2624
weighted avg       0.55      0.57      0.48      2624



/Users/pathakgaurav/anaconda3/envs/nlp_course/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
y_pred_NB= y_pred1

print(Counter(y_pred_NB))

Counter({'5': 1525, '1': 955, '8': 143, '6': 1})


# USING LSTM

In [13]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19258 unique tokens.


In [14]:
X = tokenizer.texts_to_sequences(df1['Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (13118, 250)


In [15]:
Y = pd.get_dummies(df1['Topic']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (13118, 10)


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11806, 250) (11806, 10)
(1312, 250) (1312, 10)


In [17]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 5,081,410
Trainable params: 5,081,410
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

NameError: name 'model' is not defined

In [ ]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

# #EMOTIONS AND ACTS

In [108]:
#Creating dataframe of indiviudal sentences in a conversation
n2 = {'Text':sentences,'Emotions':emo_list,'Act':act_list}
df2=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in n2.items() ]))
df2['clean'] = df2['Text'].apply(lambda x: clean_text(x))
df2['text_string'] = [','.join(map(str, l)) for l in df2['clean']]
df2['Act'].fillna(7, inplace=True)
df2['Emotions'].fillna(7, inplace=True)
df2[['text_string','Act','Emotions']] = df2[['text_string','Act','Emotions']].fillna('')
#df2['text_string'] = le.fit_transform(df2['text_string'].astype('category'))

df2

,Text,Emotions,Act,clean,text_string
0,The kitchen stinks .,2,3,"[the, kitchen, stinks, ]","the,kitchen,stinks,"
1,I'll throw out the garbage .,0,4,"[ill, throw, out, the, garbage, ]","ill,throw,out,the,garbage,"
2,"So Dick , how about getting some coffee for to...",4,3,"[so, dick, how, about, getting, some, coffee, ...","so,dick,how,about,getting,some,coffee,for,toni..."
3,Coffee ? I don ’ t honestly like that kind of ...,2,4,"[coffee, i, don, t, honestly, like, that, kind...","coffee,i,don,t,honestly,like,that,kind,of,stuff,"
4,"Come on , you can at least try a little , besi...",0,3,"[come, on, you, can, at, least, try, a, little...","come,on,you,can,at,least,try,a,little,besides,..."
5,What ’ s wrong with that ? Cigarette is the th...,1,1,"[what, s, wrong, with, that, cigarette, is, th...","what,s,wrong,with,that,cigarette,is,the,thing,..."
6,"Not for me , Dick .",0,1,"[not, for, me, dick, ]","not,for,me,dick,"
7,Are things still going badly with your housegu...,0,2,"[are, things, still, going, badly, with, your,...","are,things,still,going,badly,with,your,housegu..."
8,Getting worse . Now he ’ s eating me out of ho...,1,1,"[getting, worse, now, he, s, eating, me, out, ...","getting,worse,now,he,s,eating,me,out,of,house,..."
9,"Leo , I really think you ’ re beating around t...",0,3,"[leo, i, really, think, you, re, beating, arou...","leo,i,really,think,you,re,beating,around,the,b..."


In [119]:
df2[['text_string','Act','Emotions']] = df2[['text_string','Act','Emotions']].astype('str')
df2.dtypes

Text           object
Emotions       object
Act            object
clean          object
text_string    object
dtype: object

In [120]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(df2["text_string"],df2["Emotions"], test_size = 0.2, random_state = 1)
print(x_train1.shape)
print(y_train1.shape)
print(x_test1.shape)
print(y_test1.shape)
print(Counter(y_train1))
print(Counter(y_test1))

x_train1=x_train1.fillna("")
y_train1=y_train1.fillna("")
x_test1=x_test1.fillna("")
y_test1=y_test1.fillna("")


(82384,)
(82384,)
(20596,)
(20596,)
Counter({'0': 68383, '4': 10348, '6': 1448, '5': 949, '1': 827, '2': 293, '3': 135, '7': 1})
Counter({'0': 17189, '4': 2537, '6': 375, '5': 201, '1': 195, '2': 60, '3': 39})


(array([], dtype=int64),)

In [162]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train1, y_train1)

from sklearn.metrics import classification_report
y_pred3 = nb.predict(x_test1)

print('accuracy %s' % accuracy_score(y_pred3, y_test1))
print(classification_report(y_test1, y_pred3))

accuracy 0.834870848708487
              precision    recall  f1-score   support

           0       0.84      1.00      0.91     17189
           1       0.00      0.00      0.00       195
           2       0.00      0.00      0.00        60
           3       0.00      0.00      0.00        39
           4       0.63      0.00      0.01      2537
           5       0.00      0.00      0.00       201
           6       0.00      0.00      0.00       375

   micro avg       0.83      0.83      0.83     20596
   macro avg       0.21      0.14      0.13     20596
weighted avg       0.77      0.83      0.76     20596



/Users/pathakgaurav/anaconda3/envs/nlp_course/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# TEST

In [51]:
l_test = open("dataset-2/dialogues_text.txt").readlines()
type(l_test)

list

In [37]:
test_lst = list(map(str.strip, l_test))
test_lst=[x for x in test_lst if x]
print(test_lst[:1])

["what's up dadyo when did you get back on Twitter? Haha"]


In [66]:
m = {'Text':test_lst}
test_pd = pd.DataFrame(m)
df_test = test_pd.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
df_test['clean'] = df_test['Text'].apply(lambda x: clean_text(x))
df_test['text_string'] = [','.join(map(str, l)) for l in df_test['clean']]
df_test

,Text,clean,text_string
0,what's up dadyo when did you get back on Twitt...,"[whats, up, dadyo, when, did, you, get, back, ...","whats,up,dadyo,when,did,you,get,back,on,twitte..."
1,like 2 weeks ago and it's going as terribly as...,"[like, weeks, ago, and, its, going, as, terrib...","like,weeks,ago,and,its,going,as,terribly,as,i,..."
2,"literally never about that account, love it.","[literally, never, about, that, account, love,...","literally,never,about,that,account,love,it"
3,Answer me this fellow Apple peoples: how many ...,"[answer, me, this, fellow, apple, peoples, how...","answer,me,this,fellow,apple,peoples,how,many,t..."
4,about 50 times today. Terminal vim user.,"[about, times, today, terminal, vim, user]","about,times,today,terminal,vim,user"
5,Seems the major complaints so far are from VIM...,"[seems, the, major, complaints, so, far, are, ...","seems,the,major,complaints,so,far,are,from,vim..."
6,cmd+opt+esc is good but still available via me...,"[cmdoptesc, is, good, but, still, available, v...","cmdoptesc,is,good,but,still,available,via,menubar"
7,There was a greasy kid at highline who was bas...,"[there, was, a, greasy, kid, at, highline, who...","there,was,a,greasy,kid,at,highline,who,was,bas..."
8,I'm disgusted,"[im, disgusted]","im,disgusted"
9,he flashed us then we scored so he sadly put h...,"[he, flashed, us, then, we, scored, so, he, sa...","he,flashed,us,then,we,scored,so,he,sadly,put,h..."


In [77]:
text1 = 'i cant handle exorcist'
nb.predict_proba([text1])

array([[0.30143231, 0.01976576, 0.01461336, 0.00082576, 0.0254164 ,
        0.43228638, 0.0373156 , 0.00533549, 0.16054968, 0.00245925]])